In [1]:
import pickle
import anndata2ri
import scanpy
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

#This activates the object conversion.
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_2982348/19227154.py:9: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [2]:
#Import needed R libraries.
seurat = importr('Seurat')
base = importr('base')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [6]:
#Read Seurat object into R.
robjects.r('seurat_object = readRDS("/scratch/hnatri/PIPAC_NC50_NN20_PC20_Seurat_annotated_metadata_ncells3k_nneighbors20.rds")')

<rpy2.robjects.methods.RS4 object at 0x15550c205e40> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [7]:
#Merge layers because Seurat object versioning is wild. Convert Seurat object to Anndata.
robjects.r('seurat_object = JoinLayers(seurat_object)')
adata = robjects.r('as.SingleCellExperiment(seurat_object)')

In [8]:
#Check to make sure conversion worked and possibly cast counts to array if necessary.
adata.X = adata.X.toarray()

In [9]:
adata.X

array([[0.        , 0.        , 1.09861231, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.69314718,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [10]:
del adata.layers

In [11]:
#Write anndata to pickle file. Next move to the clustering script.
with open("/scratch/hnatri/PIPAC_NC50_NN20_PC20_Seurat_annotated_metadata_ncells3k_nneighbors20", "wb") as adata_object:
    pickle.dump(adata, adata_object)

In [12]:
adata

AnnData object with n_obs × n_vars = 1098135 × 480
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'num.blank', 'TMA', 'percent.blank', 'nCount_cell_RNA', 'nFeature_cell_RNA', 'Sample', 'ident', 'leiden_res0.5', 'leiden_res1.0', 'leiden_res1.5', 'leiden_res2.0', 'Institution', 'Patient_ID', 'Timepoint', 'Tissue', 'Location_Quadrant', 'Annotation', 'RPN', 'STUDY_SITE', 'GENDER', 'ETHNICITY', 'race', 'DISEASESITE', 'ECOG', 'ACTUALWEIGHT', 'txstartdate', 'DXHISTOLOGY', 'DXSITE', 'initialdxdate', 'LINETHERAPY', 'PRIORTHERAPYTYPE', 'priorcytoreduction', 'lastcontactdate', 'furthertxall', 'progressiondt', 'vitalstatus1', 'deathdate', 'OFFSTUDYREASON', 'offstudydate', 'OFFTXREASON', 'offtxdate', 'asascore1', 'a

In [1]:
#Write anndata to h5 file to import into R for conversion to Seurat object. Next move to commandline R for sceasy.
adata.write_h5ad(filename="/scratch/hnatri/PIPAC/nonimmune_complete_subset_clustered_NC50_NN20_PC20_AnnData.h5ad")

NameError: name 'adata' is not defined

In [14]:
adata.obs["Annotation"].unique()

array(['EpiTumor1', 'Meso', 'M8', 'F1', 'F3', 'L2', 'Mast', 'L4',
       'Stromal2', 'Endo1', 'L3', 'M1', 'M4', 'L1', 'M2', 'M6', 'Plasma1',
       'Plasma3', 'F2', 'Endo3', 'Peri', 'F4', 'B1', 'L5', 'Plasma2',
       'M7', 'Endo2', 'Stromal1', 'M5', 'B2', 'DC', 'M3', 'EpiTumor2'],
      dtype=object)